In [ ]:
import sys
import os
import arcpy

import pandas as pd
from arcgis.features import SpatialDataFrame

base_path = os.path.abspath(".")

In [ ]:
field_list = ['CO_TAZID', 'HH', 'JOB', 'accessible_jobs', 'accessible_hh', 'ato']

arr = arcpy.da.TableToNumPyArray(r'baseline.gdb\auto_baseline_summary', field_list)
baseline_auto = pd.DataFrame(arr, columns = field_list)

arr = arcpy.da.TableToNumPyArray(r'baseline.gdb\transit_baseline_summary', field_list)
baseline_transit = pd.DataFrame(arr, columns = field_list)

In [ ]:
summary_long = pd.DataFrame(columns = ['Name', 'Mode', 'hh', 'jobs', 'ato'])

vals = {
    "Name":"Baseline",
    "Mode": "Auto",
    "hh": baseline_auto['accessible_hh'].sum(),
    "jobs": baseline_auto['accessible_jobs'].sum(),
    "ato": baseline_auto['ato'].sum()
}
summary_long = summary_long.append(vals, ignore_index=True)

vals = {
    "Name": "Baseline",
    "Mode": "Transit",
    "hh": baseline_transit['accessible_hh'].sum(),
    "jobs": baseline_transit['accessible_jobs'].sum(),
    "ato": baseline_transit['ato'].sum()
}
summary_long = summary_long.append(vals, ignore_index=True)

In [ ]:
summary_long

In [ ]:
scenarios = list()

for mode in ['Driving', 'Transit', 'Cycling']:
    scenario_folder = os.path.join('scenario', mode)
    for file in os.listdir(scenario_folder):
        d = os.path.join(scenario_folder, file)
        if os.path.isdir(d) and d.endswith('.gdb'):
            if arcpy.Exists(os.path.join(d, 'scores')) == True:

                scenarios.append({
                                "name": file[:-4],
                                "mode": mode,
                                "gdb_path": d
                            })

In [ ]:
scenarios

In [ ]:
# filter to just driving for now
#scenarios = [scenario for scenario in scenarios if scenario['mode'] == mode]

In [ ]:
for scenario in scenarios:
    arr = arcpy.da.TableToNumPyArray(scenario['gdb_path'] + r'\scores_summary', field_list)
    scores = pd.DataFrame(arr, columns = field_list)
    
    mode = scenario['mode']
    baseline = baseline_auto if mode == 'Driving' else baseline_transit
    
    df = pd.merge(
        baseline, 
        scores, 
        on='CO_TAZID', 
        how="inner",
        suffixes=("_before", "_after")
    )
    df['diff_hh'] = df['accessible_hh_after'] - df['accessible_hh_before']
    df['diff_jobs'] = df['accessible_jobs_after'] - df['accessible_jobs_before']
    df['diff_ato'] = df['ato_after'] - df['ato_before']
    vals = {
        "Name": scenario['name'],
        'Mode': mode,
        "hh": df['diff_hh'].sum(),
        "jobs": df['diff_jobs'].sum(),
        "ato": df['diff_hh'].sum()
    }
    
    summary_long = summary_long.append(vals, ignore_index=True)

In [ ]:

summary_long

In [ ]:
summary_long.to_csv(r'scenario\scenario_scores.csv')

## Visualize Output

In [ ]:
arcpy.management.MakeFeatureLayer(
    os.path.join(base_path, r"shp\taz_wfrc.gdb\ATO"), 
    "taz_compare"
)

In [ ]:
arcpy.conversion.TableToTable(
    os.path.join(base_path, compare_out_file), 
    base_gdb , 
    "mod_compare"
)

In [ ]:
arcpy.management.AddJoin(
    "taz_compare", 
    "CO_TAZID", 
    "mod_compare", 
    "Origin_TAZID", 
    "KEEP_ALL"
)

Now, manually apply symbology to taz_compare layer (I experimented with creating a layer style and applying programatically), but found it easier just to manually update)
